# DataSet

In [1]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

In [2]:
cat_image_path = "../data/cifar10_images/cat"
dog_image_path = "../data/cifar10_images/dog"

In [3]:
class CustomDataset(Dataset):
    def __init__(self, dir_paths, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform

        for dir_path in dir_paths:
            label = os.path.basename(dir_path)
            for file_name in os.listdir(dir_path):
                image_path = os.path.join(dir_path, file_name)
                image = Image.open(image_path).convert('RGB')
                if self.transform:
                    image = self.transform(image)
                self.images.append(image)
                self.labels.append(label)

        self.label_to_index = {'cat': 0, 'dog': 1}
        self.labels = [self.label_to_index[label] for label in self.labels]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

In [4]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

In [5]:
dataset = CustomDataset(dir_paths=[cat_image_path, dog_image_path], transform=transform)

print(f"number of dataset: {len(dataset)}")

number of dataset: 1000


In [6]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(f"number of train dataset: {len(train_dataset)}")
print(f"number of test dataset: {len(test_dataset)}")

number of train dataset: 800
number of test dataset: 200


In [7]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for image, label in train_loader:
    print(f"data shape: {image.size()}")
    print(f"label shape: {label.size()}")
    break

data shape: torch.Size([16, 3, 32, 32])
label shape: torch.Size([16])


# HyperParameter

In [8]:
input_size = 32 * 32 * 3
hidden_size = 128
learning_rate = 0.0001
num_epochs = 10

# Model

In [9]:
import torch.nn as nn

In [10]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return x

model = SimpleNN(input_size=input_size, hidden_size=hidden_size)

In [11]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())

    return total_params

def print_model_parameters(model):
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel()} parameters")
    print(f"Total parameters: {count_parameters(model)}")

print_model_parameters(model)

fc1.weight: 393216 parameters
fc1.bias: 128 parameters
fc2.weight: 128 parameters
fc2.bias: 1 parameters
Total parameters: 393473


# Loss

In [12]:
criterion = nn.BCELoss()

# Optimizer

In [13]:
import torch.optim as optim

In [14]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Train

In [15]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.view(images.size(0), -1)
        labels = labels.float().view(-1, 1)

        outputs = model(images)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')


Epoch 1/10, Loss: 0.6915
Epoch 2/10, Loss: 0.6912
Epoch 3/10, Loss: 0.6910
Epoch 4/10, Loss: 0.6907
Epoch 5/10, Loss: 0.6905
Epoch 6/10, Loss: 0.6903
Epoch 7/10, Loss: 0.6900
Epoch 8/10, Loss: 0.6898
Epoch 9/10, Loss: 0.6896
Epoch 10/10, Loss: 0.6893


# Predict

In [16]:
import torch

In [17]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(images.size(0), -1)
        labels = labels.float().view(-1, 1)

        outputs = model(images)

        predicted = (outputs > 0.5).float()

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Set Accuracy: {accuracy * 100:.2f}%")

Test Set Accuracy: 61.50%
